<h1> Deploying and predicting with model </h1>

This notebook illustrates:
<ol>
<li> Deploying model
<li> Predicting with model
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-e6c6fe8dd7979408'
PROJECT = 'qwiklabs-gcp-e6c6fe8dd7979408'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.7'  

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
  # copy canonical model if you didn't do previous notebook
  gsutil -m cp -R gs://cloud-training-demos/babyweight/trained_model gs://${BUCKET}/babyweight/trained_model
fi

<h2> Deploy trained model </h2>
<p>
Deploying the trained model to act as a REST web service is a simple gcloud call.

In [4]:
%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/

gs://qwiklabs-gcp-e6c6fe8dd7979408/babyweight/trained_model/export/exporter/
gs://qwiklabs-gcp-e6c6fe8dd7979408/babyweight/trained_model/export/exporter/1536188147/


In [5]:
%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying babyweight ml_on_gcp from gs://qwiklabs-gcp-e6c6fe8dd7979408/babyweight/trained_model/export/exporter/1536188147/ ... this will take a few minutes


Created ml engine model [projects/qwiklabs-gcp-e6c6fe8dd7979408/models/babyweight].
Creating version (this might take a few minutes)......
...............................................................................................................................................................................................................................................................................................................................................................................................................................................done.


<h2> Use model to predict (online prediction) </h2>
<p>
Send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

In [6]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = 'babyweight'
MODEL_VERSION = 'ml_on_gcp'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'key': 'b1',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 39
    },
    {
      'key': 'g1',
      'is_male': 'False',
      'mother_age': 29.0,
      'plurality': 'Single(1)',
      'gestation_weeks': 38
    },
    {
      'key': 'b2',
      'is_male': 'True',
      'mother_age': 26.0,
      'plurality': 'Triplets(3)',
      'gestation_weeks': 39
    },
    {
      'key': 'u1',
      'is_male': 'Unknown',
      'mother_age': 29.0,
      'plurality': 'Multiple(2+)',
      'gestation_weeks': 38
    },
  ]
}
response = requests.post(api, json=data, headers=headers)
print response.content

{"predictions": [{"predictions": [0.6718437671661377], "key": ["b1"]}, {"predictions": [0.28848713636398315], "key": ["g1"]}, {"predictions": [0.24402552843093872], "key": ["b2"]}, {"predictions": [0.2945956289768219], "key": ["u1"]}]}


The predictions for the four instances were: 7.66, 7.22, 6.32 and 6.19 pounds respectively when I ran it (your results might be different).

<h2> Use model to predict (batch prediction) </h2>
<p>
Batch prediction is commonly used when you thousands to millions of predictions.
Create a file withe one instance per line and submit using gcloud.

In [7]:
%writefile inputs.json
{"key": "b1", "is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"key": "g1", "is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Overwriting inputs.json


In [8]:
%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ml-engine jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=babyweight --version=ml_on_gcp

jobId: babypred_180905_232429
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
/ [1 files][  204.0 B/  204.0 B]                                                
Operation completed over 1 objects/204.0 B.                                      
CommandException: 1 files/objects could not be removed.
Job [babypred_180905_232429] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe babypred_180905_232429

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs babypred_180905_232429


Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License